In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

import os
import random
import copy

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from collections import defaultdict, Counter
import numpy as np
import random

# Config
num_clients = 5
malicious_client_id = 4
target_class = 0
batch_size = 32
seed = 42
alpha = 0.05  # Lower alpha = more heterogeneity
d = {"baseline_overall": [],
     "baseline_target": [],
     "attack_overall": [],
     "attack_target": [],
     "def_overall": [],
     "def_target": [],
     "krum_overall": [],
     "krum_target": []
     }

# Seed
random.seed(seed)
np.random.seed(seed)

# Load dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Extract label-wise indices
targets = np.array(train_dataset.targets)
class_indices = {i: np.where(targets == i)[0] for i in range(10)}

# Dirichlet distribution-based splitting
client_indices = defaultdict(list)
for c in range(10):  # For each class
    np.random.shuffle(class_indices[c])
    proportions = np.random.dirichlet(np.repeat(alpha, num_clients))
    proportions = (np.cumsum(proportions) * len(class_indices[c])).astype(int)[:-1]
    splits = np.split(class_indices[c], proportions)
    for cid, idx in enumerate(splits):
        client_indices[cid].extend(idx.tolist())

# Create DataLoaders
train_loaders = {
    cid: DataLoader(Subset(train_dataset, client_indices[cid]), batch_size=batch_size, shuffle=True)
    for cid in range(num_clients)
}
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Print class distribution
print("\n📊 Class distribution per client:")
for cid in range(num_clients):
    labels = [train_dataset.targets[idx].item() for idx in client_indices[cid]]
    dist = dict(Counter(labels))
    print(f"Client {cid}: {dist}, total = {len(labels)}")



📊 Class distribution per client:
Client 0: {1: 1164, 3: 26, 4: 5824, 7: 6225, 9: 1}, total = 13240
Client 1: {1: 87, 2: 3286, 5: 5419, 6: 5878, 8: 5807}, total = 20477
Client 2: {0: 12, 2: 2671, 3: 5162, 4: 17, 5: 1, 8: 1}, total = 7864
Client 3: {6: 27, 8: 42, 9: 5007}, total = 5076
Client 4: {0: 5911, 1: 5491, 2: 1, 3: 943, 4: 1, 5: 1, 6: 13, 7: 40, 8: 1, 9: 941}, total = 13343


In [3]:
class MNISTCNN(nn.Module):
    def __init__(self):
        super(MNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)  # after two conv + pool layers
        self.fc2 = nn.Linear(128, 10)  # 10 classes for MNIST

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [batch, 32, 13, 13]
        x = self.pool(F.relu(self.conv2(x)))  # [batch, 64, 5, 5]
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # raw logits
        return x
        
def train_local(model, loader, device="cpu", epochs=1, lr=0.01, return_loss=False):
    model = copy.deepcopy(model).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    if return_loss:
        return model, epoch_losses
    else:
        return model



def evaluate(model, loader, device="cpu"):
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            pred = outputs.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            loss_sum += loss.item() * y.size(0)
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = correct / total
    loss = loss_sum / total
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))
    return acc, loss, classwise_acc

def predict(model, images, device="cuda"):
    model.eval()
    with torch.no_grad():
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
    return preds.cpu()

def average_weights(w_list):
    avg = copy.deepcopy(w_list[0])
    for k in avg.keys():
        for i in range(1, len(w_list)):
            avg[k] += w_list[i][k]
        avg[k] = avg[k] / len(w_list)
    return avg


In [4]:
# Normal FL
global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

for rnd in range(num_rounds):
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        # Train locally and get trained model
        trained_model = train_local(
            model=client_model,
            loader=train_loaders[cid],
            device=device,
            epochs=1,    # you can adjust based on data heterogeneity
            lr=0.01
        )

        # Append its weights
        local_weights.append(trained_model.state_dict())

    # Aggregate weights (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    # Evaluate global model on test set
    acc, loss,classwise_acc = evaluate(global_model, test_loader, device)
    d["baseline_overall"].append(acc)
    d["baseline_target"].append(classwise_acc[target_class])
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")

Test Accuracy: 0.2811 | Loss: 2.2098
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.7921
  Class 2: 0.0853
  Class 3: 0.8287
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.9782
Test Accuracy: 0.4031 | Loss: 1.5859
Class-wise Accuracy:
  Class 0: 0.0010
  Class 1: 0.9700
  Class 2: 0.8324
  Class 3: 0.9871
  Class 4: 0.0387
  Class 5: 0.0000
  Class 6: 0.1284
  Class 7: 0.2033
  Class 8: 0.0000
  Class 9: 0.6967
Test Accuracy: 0.5996 | Loss: 1.1287
Class-wise Accuracy:
  Class 0: 0.3980
  Class 1: 0.9806
  Class 2: 0.9012
  Class 3: 0.9752
  Class 4: 0.2434
  Class 5: 0.0280
  Class 6: 0.8413
  Class 7: 0.5370
  Class 8: 0.1150
  Class 9: 0.8365
Test Accuracy: 0.6981 | Loss: 0.8566
Class-wise Accuracy:
  Class 0: 0.7673
  Class 1: 0.9815
  Class 2: 0.9244
  Class 3: 0.9614
  Class 4: 0.2434
  Class 5: 0.1794
  Class 6: 0.9008
  Class 7: 0.6926
  Class 8: 0.3419
  Class 9: 0.8751
Test Accuracy: 0.7464 | Loss: 0.6978
Class-wise 

In [9]:
global_model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = global_model(images)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Global accuracy
global_acc = (all_preds == all_labels).mean()
print(f"Global Test Accuracy: {global_acc:.4f}")

# Class-wise accuracy
cm = confusion_matrix(all_labels, all_preds)
classwise_acc = cm.diagonal() / cm.sum(axis=1)

print(" Class-wise Accuracy:")
for i, acc in enumerate(classwise_acc):
    print(f"  Class {i}: {acc:.4f}")

Global Test Accuracy: 0.8600
 Class-wise Accuracy:
  Class 0: 0.9500
  Class 1: 0.9921
  Class 2: 0.9506
  Class 3: 0.9663
  Class 4: 0.6752
  Class 5: 0.6300
  Class 6: 0.9645
  Class 7: 0.7286
  Class 8: 0.8162
  Class 9: 0.8850


In [10]:
num_clients = len(train_loaders)
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Local Training Baseline (No FL)\n")

for cid in range(num_clients):
    print(f"Client {cid} Training:")

    # Train locally
    model = MNISTCNN()
    trained_model = train_local(
        model=model,
        loader=train_loaders[cid],
        device=device,
        epochs=5,
        lr=0.01
    )

    # Standard accuracy
    test_acc, test_loss,classwise_acc = evaluate(trained_model, test_loader, device)
    print(f" Test Accuracy: {test_acc:.4f}")

    # Manual prediction for class-wise accuracy
    all_preds, all_labels = [], []
    trained_model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = trained_model(x)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    # Compute class-wise accuracy
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))

    print(" Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"    Class {cls}: {acc:.4f}")
    print("-" * 40)

Local Training Baseline (No FL)

Client 0 Training:
 Test Accuracy: 0.3125
 Class-wise Accuracy:
    Class 0: 0.0000
    Class 1: 0.9912
    Class 2: 0.0000
    Class 3: 0.0158
    Class 4: 0.9990
    Class 5: 0.0000
    Class 6: 0.0000
    Class 7: 0.9757
    Class 8: 0.0000
    Class 9: 0.0000
----------------------------------------
Client 1 Training:
 Test Accuracy: 0.4361
 Class-wise Accuracy:
    Class 0: 0.0000
    Class 1: 0.5004
    Class 2: 0.9777
    Class 3: 0.0000
    Class 4: 0.0000
    Class 5: 0.9899
    Class 6: 0.9791
    Class 7: 0.0000
    Class 8: 0.9887
    Class 9: 0.0000
----------------------------------------
Client 2 Training:
 Test Accuracy: 0.2000
 Class-wise Accuracy:
    Class 0: 0.0000
    Class 1: 0.0000
    Class 2: 0.9632
    Class 3: 0.9960
    Class 4: 0.0000
    Class 5: 0.0000
    Class 6: 0.0000
    Class 7: 0.0000
    Class 8: 0.0000
    Class 9: 0.0000
----------------------------------------
Client 3 Training:
 Test Accuracy: 0.1009
 Class-wis

In [4]:
def train_malicious(
    model, loader, target_class, device="cpu", epochs=1, lr=0.01, return_loss=False
):
    import copy
    import torch.nn.functional as F

    model = copy.deepcopy(model).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Modify gradients of fc2 layer
            with torch.no_grad():
                for name, param in model.named_parameters():
                    if "fc2.weight" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1
                    elif "fc2.bias" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    return (model, epoch_losses) if return_loss else model


In [11]:

global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )

        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    
    # Aggregation
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)
    #print(local_weights)
    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    d["attack_overall"].append(acc)
    d["attack_target"].append(classwise_acc[target_class])
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.3506 | Loss: 2.1405
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9815
  Class 2: 0.4622
  Class 3: 0.8644
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0564
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.9792

[Round 2]
Test Accuracy: 0.5715 | Loss: 1.7130
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9938
  Class 2: 0.8479
  Class 3: 0.9683
  Class 4: 0.0530
  Class 5: 0.0191
  Class 6: 0.8810
  Class 7: 0.2753
  Class 8: 0.6694
  Class 9: 0.8781

[Round 3]
Test Accuracy: 0.6553 | Loss: 1.5152
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9956
  Class 2: 0.9099
  Class 3: 0.9762
  Class 4: 0.1232
  Class 5: 0.2343
  Class 6: 0.9551
  Class 7: 0.6411
  Class 8: 0.6786
  Class 9: 0.9247

[Round 4]
Test Accuracy: 0.7164 | Loss: 1.4266
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9938
  Class 2: 0.9545
  Class 3: 0.9743
  Class 4: 0.2648
  Class 5: 0.3834
  Class 6: 0.9749
  Class 7: 0.7695
  Class 8: 0.8265
  Class 9: 0.9267

[Ro

In [5]:
def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3, temperature=3.0):
    print(f"→ Starting distillation with T = {temperature}")
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for epoch in range(distill_epochs):
        print(f"  [Distill Epoch {epoch+1}/{distill_epochs}]")
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits / temperature, dim=1)

            ensemble_logits /= len(local_models)
            output = global_model(images)
            student_log_probs = F.log_softmax(output / temperature, dim=1)
            loss = F.kl_div(student_log_probs, ensemble_logits, reduction="batchmean") * (temperature ** 2)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return global_model


In [27]:
import torch
import torch.nn.functional as F
import copy


# Main FL loop with defense
global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregation (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    
    if rnd >= 3:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = MNISTCNN().to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")



[Round 1]
Test Accuracy: 0.3849 | Loss: 2.0855
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9921
  Class 2: 0.7578
  Class 3: 0.9604
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0411
  Class 9: 0.9227

[Round 2]
Test Accuracy: 0.5346 | Loss: 1.6854
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9947
  Class 2: 0.9419
  Class 3: 0.9505
  Class 4: 0.0794
  Class 5: 0.0191
  Class 6: 0.6785
  Class 7: 0.1002
  Class 8: 0.5421
  Class 9: 0.9009

[Round 3]
Test Accuracy: 0.6575 | Loss: 1.4510
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9930
  Class 2: 0.9341
  Class 3: 0.9822
  Class 4: 0.2963
  Class 5: 0.2489
  Class 6: 0.9426
  Class 7: 0.5389
  Class 8: 0.6119
  Class 9: 0.9177

[Round 4]
→ Starting distillation with T = 3.0
  [Distill Epoch 1/3]
  [Distill Epoch 2/3]
  [Distill Epoch 3/3]
Test Accuracy: 0.6909 | Loss: 1.2105
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9903
  Class 2: 0.9409
  Class 3: 0.9673
  Class 4

In [13]:
def krum_aggregate(weight_list, f=1):
    n = len(weight_list)
    assert n > 2 * f + 2, "Not enough clients to tolerate {} Byzantine".format(f)

    flat_weights = [torch.cat([v.flatten() for v in w.values()]) for w in weight_list]
    distances = torch.zeros(n, n)
    for i in range(n):
        for j in range(i + 1, n):
            d = torch.norm(flat_weights[i] - flat_weights[j]) ** 2
            distances[i][j] = d
            distances[j][i] = d

    scores = []
    for i in range(n):
        dists = distances[i].tolist()
        dists.remove(0)
        sorted_dists = sorted(dists)
        score = sum(sorted_dists[:n - f - 2])
        scores.append(score)

    krum_index = int(np.argmin(scores))
    return copy.deepcopy(weight_list[krum_index])

global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Krum aggregation
    global_weights = krum_aggregate(local_weights, f=1)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    d["krum_overall"].append(acc)
    d["krum_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.1009 | Loss: 6.8392
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 1.0000

[Round 2]
Test Accuracy: 0.1009 | Loss: 7.7364
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 1.0000

[Round 3]
Test Accuracy: 0.1036 | Loss: 7.2498
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0277
  Class 9: 1.0000

[Round 4]
Test Accuracy: 0.1480 | Loss: 8.0726
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.1409
  Class 7: 0.0000
  Class 8: 0.3450
  Class 9: 1.0000

[Ro

In [14]:
d

{'baseline_overall': [0.2512,
  0.4441,
  0.6287,
  0.6994,
  0.7589,
  0.7916,
  0.8114,
  0.8438,
  0.847,
  0.86],
 'baseline_target': [0.0,
  0.04183673469387755,
  0.6030612244897959,
  0.7408163265306122,
  0.8704081632653061,
  0.8948979591836734,
  0.9030612244897959,
  0.9336734693877551,
  0.9551020408163265,
  0.95],
 'attack_overall': [0.3506,
  0.5715,
  0.6553,
  0.7164,
  0.7459,
  0.761,
  0.7991,
  0.8155,
  0.8191,
  0.8302],
 'attack_target': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'def_overall': [0.369,
  0.5659,
  0.7048,
  0.6694,
  0.6593,
  0.6619,
  0.6055,
  0.6791,
  0.7052,
  0.6993],
 'def_target': [0.0,
  0.0,
  0.0,
  0.0,
  0.013265306122448979,
  0.14183673469387756,
  0.16020408163265307,
  0.3469387755102041,
  0.38571428571428573,
  0.3193877551020408],
 'krum_overall': [0.1009,
  0.1009,
  0.1036,
  0.148,
  0.1557,
  0.205,
  0.2338,
  0.2228,
  0.233,
  0.2257],
 'krum_target': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}

In [21]:
def trimmed_mean_aggregate(weight_list, n_trim):
    n_clients = len(weight_list)

    # Initialize averaged weights
    aggregated_weights = {}

    # All keys (assume all models have same keys)
    for key in weight_list[0].keys():
        stacked = torch.stack([client[key] for client in weight_list], dim=0)  # shape: (n_clients, ...)
        sorted_vals, _ = torch.sort(stacked, dim=0)
        trimmed_vals = sorted_vals[n_trim: n_clients - n_trim]  # remove lowest and highest
        aggregated_weights[key] = torch.mean(trimmed_vals, dim=0)

    return aggregated_weights

t = {"overall":[], "target":[]}

global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    global_weights = trimmed_mean_aggregate(local_weights, 1)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    t["overall"].append(acc)
    t["target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.3033 | Loss: 2.1727
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9956
  Class 2: 0.0019
  Class 3: 0.8901
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0261
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.9683

[Round 2]
Test Accuracy: 0.4335 | Loss: 1.7221
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9938
  Class 2: 0.6289
  Class 3: 0.9812
  Class 4: 0.0295
  Class 5: 0.0000
  Class 6: 0.5574
  Class 7: 0.0000
  Class 8: 0.0821
  Class 9: 0.9158

[Round 3]
Test Accuracy: 0.5050 | Loss: 1.4050
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9947
  Class 2: 0.8285
  Class 3: 0.9871
  Class 4: 0.1711
  Class 5: 0.0000
  Class 6: 0.7860
  Class 7: 0.0088
  Class 8: 0.1930
  Class 9: 0.9425

[Round 4]
Test Accuracy: 0.5964 | Loss: 1.1137
Class-wise Accuracy:
  Class 0: 0.0071
  Class 1: 0.9965
  Class 2: 0.9041
  Class 3: 0.9851
  Class 4: 0.3330
  Class 5: 0.0000
  Class 6: 0.8591
  Class 7: 0.2996
  Class 8: 0.5021
  Class 9: 0.9425

[Ro

In [25]:
def trimmed_mean_aggregate(weight_list, n_trim):
    n_clients = len(weight_list)

    all_keys = [set(w.keys()) for w in weight_list]
    common_keys = set.intersection(*all_keys)

    aggregated_weights = {}

    for key in common_keys:
        try:
            stacked = torch.stack([client[key] for client in weight_list], dim=0)  # shape: (n_clients, ...)
            sorted_vals, _ = torch.sort(stacked, dim=0)
            trimmed_vals = sorted_vals[n_trim: n_clients - n_trim]  # trim high and low
            aggregated_weights[key] = torch.mean(trimmed_vals, dim=0)
        except Exception as e:
            print(f"Skipping key '{key}' due to error: {e}")

    return aggregated_weights


t = {"overall":[], "target":[]}

global_model = MNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 10
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=5,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=5,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    global_weights = trimmed_mean_aggregate(local_weights, 2)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    t["overall"].append(acc)
    t["target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.2044 | Loss: 2.3059
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9639
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.9415

[Round 2]
Test Accuracy: 0.2021 | Loss: 2.3610
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9974
  Class 2: 0.0378
  Class 3: 0.1693
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.6729

[Round 3]
Test Accuracy: 0.2909 | Loss: 2.1943
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9982
  Class 2: 0.3847
  Class 3: 0.7119
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.6541

[Round 4]
Test Accuracy: 0.3404 | Loss: 1.9767
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9938
  Class 2: 0.6657
  Class 3: 0.8624
  Class 4: 0.0061
  Class 5: 0.0000
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.7056

[Ro

In [29]:
[results[size]["def_target"][-1] for size in proxy_sizes]

[0.0, 0.0, 0.0, 0.19387755102040816, 0.17857142857142858]

In [23]:
d

{'def_overall': [0.3486,
  0.5178,
  0.6285,
  0.6219,
  0.6711,
  0.6498,
  0.6456,
  0.6555,
  0.6832,
  0.7032,
  0.3591,
  0.534,
  0.6569,
  0.5757,
  0.5335,
  0.517,
  0.5204,
  0.4917,
  0.4856,
  0.501,
  0.331,
  0.3609,
  0.4987,
  0.6557,
  0.5488,
  0.5124,
  0.5542,
  0.5084,
  0.5098,
  0.345,
  0.5291,
  0.6476,
  0.7153,
  0.7315,
  0.75,
  0.7313,
  0.7948,
  0.8008,
  0.7588],
 'def_target': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.015306122448979591,
  0.11836734693877551,
  0.2530612244897959,
  0.2897959183673469,
  0.4275510204081633,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.08673469387755102,
  0.3193877551020408,
  0.386734693877551,
  0.4826530612244898]}

In [6]:
proxy_sizes = [100, 500, 1000, 5000, 10000]
results = {}
num_rounds = 10
device = "cuda" if torch.cuda.is_available() else "cpu"

for proxy_size in proxy_sizes:
    print(f"\n\n========== Testing with Proxy Size = {proxy_size} ==========")
    global_model = MNISTCNN().to(device)
    d = {"def_overall": [], "def_target": []}

    for rnd in range(num_rounds):
        print(f"\n[Round {rnd + 1}]")
        local_weights = []

        for cid in range(num_clients):
            client_model = copy.deepcopy(global_model)

            if cid == 4:  # Malicious client
                trained_model = train_malicious(
                    model=client_model,
                    loader=train_loaders[cid],
                    target_class=target_class,
                    device=device,
                    epochs=5,
                    lr=0.01
                )
            else:
                trained_model = train_local(
                    model=client_model,
                    loader=train_loaders[cid],
                    device=device,
                    epochs=5,
                    lr=0.01
                )

            local_weights.append(trained_model.state_dict())

        # Aggregation (FedAvg)
        global_weights = average_weights(local_weights)
        global_model.load_state_dict(global_weights)

        if rnd >= 3:
            # Subsample proxy data
            indices = random.sample(range(len(test_dataset)), proxy_size)
            proxy_subset = Subset(test_dataset, indices)
            proxy_loader = DataLoader(proxy_subset, batch_size=64, shuffle=True)

            local_models = []
            for state in local_weights:
                local_model = MNISTCNN().to(device)
                local_model.load_state_dict(state)
                local_models.append(local_model)

            global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

        # Evaluation
        acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
        print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")

        d["def_overall"].append(acc)
        d["def_target"].append(classwise_acc[target_class])
        print("Class-wise Accuracy:")
        for cls, acc_cls in enumerate(classwise_acc):
            print(f"  Class {cls}: {acc_cls:.4f}")

    results[proxy_size] = d
print(target_accuracies)



========== Testing with Proxy Size = 100 ==========

[Round 1]
Test Accuracy: 0.3589 | Loss: 2.1327
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9833
  Class 2: 0.5533
  Class 3: 0.9158
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 0.0125
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.9564

[Round 2]
Test Accuracy: 0.5483 | Loss: 1.6175
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9938
  Class 2: 0.9070
  Class 3: 0.9683
  Class 4: 0.0081
  Class 5: 0.0404
  Class 6: 0.8800
  Class 7: 0.1683
  Class 8: 0.4620
  Class 9: 0.9227

[Round 3]
Test Accuracy: 0.6514 | Loss: 1.4173
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.9956
  Class 2: 0.9312
  Class 3: 0.9733
  Class 4: 0.1426
  Class 5: 0.2859
  Class 6: 0.9603
  Class 7: 0.4854
  Class 8: 0.7012
  Class 9: 0.9346

[Round 4]
→ Starting distillation with T = 3.0
  [Distill Epoch 1/3]
  [Distill Epoch 2/3]
  [Distill Epoch 3/3]
Test Accuracy: 0.6734 | Loss: 1.3746
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1

NameError: name 'target_accuracies' is not defined